In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfTransformer
import os, os.path

## 1) Import Document

In [2]:
categories = ['soc.religion.christian']
twenty_train = fetch_20newsgroups(subset='train',
                                  categories=categories,
                                  shuffle=True, random_state=42)


In [3]:
print(len(twenty_train.data)) #nbr de document
text = twenty_train.data[0] #premier document
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())
# Test the tokenizer on a piece of text
sentences = "Mr. James told me Dr. Brown is not available today. I will try tomorrow."
 
print(tokenizer.tokenize(sentences))
# ['Mr. James told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']
 

599
['Mr.', 'James told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']


In [4]:
X = tokenizer.tokenize(text)
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                                 token_pattern=r'\b\w+\b', min_df=1)
X_train_counts = bigram_vectorizer.fit_transform(X)
X_train_counts.shape
print(X_train_counts.shape) #nbr documents * nombre de mots possibles
print(type(X_train_counts))
#print(X_train_counts)

(79, 1731)
<class 'scipy.sparse.csr.csr_matrix'>


In [5]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf)

  (0, 2)	0.14002800840280097
  (0, 3)	0.14002800840280097
  (0, 26)	0.14002800840280097
  (0, 27)	0.14002800840280097
  (0, 30)	0.14002800840280097
  (0, 31)	0.14002800840280097
  (0, 37)	0.14002800840280097
  (0, 38)	0.14002800840280097
  (0, 44)	0.14002800840280097
  (0, 45)	0.14002800840280097
  (0, 104)	0.14002800840280097
  (0, 105)	0.14002800840280097
  (0, 190)	0.14002800840280097
  (0, 196)	0.14002800840280097
  (0, 346)	0.14002800840280097
  (0, 347)	0.14002800840280097
  (0, 473)	0.14002800840280097
  (0, 474)	0.14002800840280097
  (0, 561)	0.14002800840280097
  (0, 564)	0.14002800840280097
  (0, 614)	0.14002800840280097
  (0, 626)	0.14002800840280097
  (0, 627)	0.14002800840280097
  (0, 714)	0.14002800840280097
  (0, 717)	0.14002800840280097
  :	:
  (77, 1182)	0.10976425998969035
  (77, 1191)	0.10976425998969035
  (77, 1192)	0.10976425998969035
  (77, 1366)	0.10976425998969035
  (77, 1367)	0.10976425998969035
  (77, 1372)	0.10976425998969035
  (77, 1373)	0.10976425998969035


## 2) Submodular Function to maximize

In [6]:
def covdiv_F(S, V, cluster, lambda1=0.5):
    if len(S)==0:
        return(0)
    return(cov_L(S, V) + lambda1*div_R(S, V, cluster))

def cov_L(S, V, alpha=0.8):
    res = 0
    y= V[S]
    for x in V:
        res1 = cosine_similarity(x.reshape(1, -1), y).sum()
        res2 = alpha*cosine_similarity(x.reshape(1, -1), V).sum()
        res += min(res1, res2)
    return(res)

def div_R(S, V, cluster, alpha=0.8):
    K = len(cluster)
    res = 0
    for k in range(K):
        A = list(set(S) & set(cluster[k]))
        S_inter_Pk = V[A]
        res1 = 0
        for j in S_inter_Pk:
            res1 += cosine_similarity(j.reshape(1, -1), V).sum()
        res += np.sqrt(res1)
    return(res)

In [7]:
# Clustering for div_R
from sklearn.cluster import KMeans
X = X_train_tf.toarray()
# Initializing KMeans
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters)
# Fitting with inputs
kmeans = kmeans.fit(X)
# Predicting the clusters
labels = kmeans.predict(X)
# Getting the cluster centers
C = kmeans.cluster_centers_
cluster = {k:[] for k in range(n_clusters)}
for i in range(len(labels)):
    cluster[labels[i]].append(i)

## 3) Algorithm

In [9]:
#def g_1(f, G, u, c, r):
#    return((f(G + [u])-f(G))/(c([u]))**r)

def f_sub(S):
    return(covdiv_F(S, X_train_tf, cluster))

def cost(S):
    return(len(S))

B = 3

V = list(range(77))

def greedy_submodular(f, V, c):
    r=1
    G = []
    U = list(range(V.shape[0]))
    cost = 0
    while len(U)!=0 and B-cost>=1:
        L = [(f(G + [u])-f(G))/(c([u]))**r for u in U]
        k = U[np.array(L).argmax()]
        if cost + c([k]) <= B: # and f(G + [k])- f(G) >= 0: 
            G += [k]
            cost += c([k])
        U.remove(k)
        print(G)
    L = [f([i]) for i in V if c([i])<B]
    v = np.array(L).argmax()
    if f(G)>f([v]):
        return(G)
    else:
        return(v)

In [10]:
greedy_submodular(f_sub, X_train_tf, cost)

[46]
[46, 58]
[46, 58, 69]


[46, 58, 69]

## 4) New dataset : a study
### Let's first define the metric : ROUGE-N

In [23]:
def rouge_n(S, X_count, gold_sum_count):
    '''
    S (list) : list of the index of the selected sentences S
    
    X_count (sparse.matrix) : sparse matrix of the studied document.  
        X_count[k,s] : number of times the bigram s occurs in the sentence k
        
    gold_sum_count (list of sparse matrix)  for each sparse matrix,  
        gold_sum_count[k][i,j] : number of times the bigram j occurs in the sentence 
        in the summary k'''
    X_count_S_1 = X_count[S].sum(axis=0) #delete the sentence structure
    # X_count_S_1[k,s] : number of times the bigram s occurs in the summary S
    gold_summ_count_1 = [k.sum(axis=0) for k in gold_sum_count] #delete the sentence structure
    num = 0
    denom = 0
    for i in range(len(gold_summ_count_1)):
        for j in range(X_count_S_1.shape[1]):
            r_ei = gold_summ_count_1[i][0,j]
            if r_ei != 0:
                c_es = X_count_S_1[0,j]
                num += min(c_es, r_ei)
                denom += r_ei
    res = num/denom
    return(res)

### a) Import Data (opinosis)

In [12]:
path = 'OpinosisDataset1.0_0/topics/accuracy_garmin_nuvi_255W_gps.txt.data'
text = open(path,'r')
text = text.read()

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())

X = tokenizer.tokenize(text)
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                               token_pattern=r'\b\w+\b', min_df=1)
X_train_counts = bigram_vectorizer.fit_transform(X)
X_train_counts.shape
print(X_train_counts.shape) #nbr documents * nombre de mots possibles
print(type(X_train_counts))
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf)

(65, 1328)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 53)	0.30151134457776363
  (0, 130)	0.30151134457776363
  (0, 142)	0.30151134457776363
  (0, 596)	0.30151134457776363
  (0, 619)	0.30151134457776363
  (0, 1227)	0.6030226891555273
  (0, 1228)	0.30151134457776363
  (0, 1234)	0.30151134457776363
  (1, 53)	0.15617376188860607
  (1, 59)	0.15617376188860607
  (1, 266)	0.15617376188860607
  (1, 268)	0.15617376188860607
  (1, 333)	0.15617376188860607
  (1, 339)	0.15617376188860607
  (1, 417)	0.15617376188860607
  (1, 420)	0.15617376188860607
  (1, 424)	0.15617376188860607
  (1, 431)	0.15617376188860607
  (1, 449)	0.15617376188860607
  (1, 454)	0.15617376188860607
  (1, 473)	0.15617376188860607
  (1, 582)	0.15617376188860607
  (1, 583)	0.15617376188860607
  (1, 737)	0.15617376188860607
  (1, 740)	0.15617376188860607
  :	:
  (63, 1128)	0.12216944435630522
  (63, 1133)	0.12216944435630522
  (63, 1147)	0.12216944435630522
  (63, 1255)	0.12216944435630522
  (63, 1256)	0.12216944435630522
  (64,

### b) Clustering

In [13]:
# Clustering for div_R
from sklearn.cluster import KMeans
X = X_train_tf.toarray()
# Initializing KMeans
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters)
# Fitting with inputs
kmeans = kmeans.fit(X)
# Predicting the clusters
labels = kmeans.predict(X)
# Getting the cluster centers
C = kmeans.cluster_centers_
cluster = {k:[] for k in range(n_clusters)}
for i in range(len(labels)):
    cluster[labels[i]].append(i)

### c) Experiment

In [14]:
greedy_submodular(f_sub, X_train_tf, cost)

[35]
[35, 55]
[35, 55, 49]


[35, 55, 49]

In [15]:
path = 'OpinosisDataset1.0_0/summaries-gold/accuracy_garmin_nuvi_255W_gps'
K = len([name for name in os.listdir(path)])
print(K)
gold_summ_txt = []
gold_summ_count = []
for i in range(1,K+1):
    text2 = open(path + '/accuracy_garmin_nuvi_255W_gps.{}.gold'.format(i),'r')
    txt = text2.read()
    print(tokenizer.tokenize(txt))
    gold_summ_txt.append(txt)
    a = tokenizer.tokenize(txt)
    if len(a)!=0:
        gold_summ_count.append(bigram_vectorizer.transform((a)))
gold_summ_count

5
['This unit is generally quite accurate.', 'Set-up and usage are considered to be very easy.', 'The maps can be updated, and tend to be reliable.']
['The Garmin seems to be generally very accurate.', "It's easy to use with an intuitive interface."]
['It is very accurate, even in destination time.']
['Very accurate with travel and destination time.', 'Negatives are not accurate with speed limits and rural roads.']
['Its accurate, fast and its simple operations make this a for sure buy.']


[<3x1328 sparse matrix of type '<class 'numpy.int64'>'
 	with 29 stored elements in Compressed Sparse Row format>,
 <2x1328 sparse matrix of type '<class 'numpy.int64'>'
 	with 25 stored elements in Compressed Sparse Row format>,
 <1x1328 sparse matrix of type '<class 'numpy.int64'>'
 	with 13 stored elements in Compressed Sparse Row format>,
 <2x1328 sparse matrix of type '<class 'numpy.int64'>'
 	with 22 stored elements in Compressed Sparse Row format>,
 <1x1328 sparse matrix of type '<class 'numpy.int64'>'
 	with 8 stored elements in Compressed Sparse Row format>]